In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy 
import numpy as np

import time 
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
TEXT = data.Field(lower = True)
UP_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

In [4]:
fields = (("text", TEXT), ("udtags", UP_TAGS), ("ptbtags", PTB_TAGS))

In [5]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

In [6]:
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [7]:
MIN_FREQ = 2
TEXT.build_vocab(train_data,
                 min_freq=MIN_FREQ,
                 vectors="glove.6B.100d",
                 unk_init=torch.Tensor.normal_)

In [8]:
UP_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]


In [10]:
print(UP_TAGS.vocab.freqs.most_common())

[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [11]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
    
    return tag_counts_percentages

In [12]:
for tag, count, percent in tag_percentage(UP_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

NOUN		34781		17.0%
PUNCT		23679		11.6%
VERB		23081		11.3%
PRON		18577		 9.1%
ADP		17638		 8.6%
DET		16285		 8.0%
PROPN		12946		 6.3%
ADJ		12477		 6.1%
AUX		12343		 6.0%
ADV		10548		 5.2%
CCONJ		6707		 3.3%
PART		5567		 2.7%
NUM		3999		 2.0%
SCONJ		3843		 1.9%
X		847		 0.4%
INTJ		688		 0.3%
SYM		599		 0.3%


In [13]:
BATCH_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

In [183]:
class BiLSTMPOSTagger(nn.Module):
    
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):
        
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0
                           )
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidde_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
        
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        prediction = self.fc(self.dropout(outputs))
        
        return prediction

In [184]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UP_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM,
                        EMBEDDING_DIM,
                        HIDDEN_DIM,
                        OUTPUT_DIM,
                        N_LAYERS,
                        BIDIRECTIONAL,
                        DROPOUT,
                        PAD_IDX
                       )

In [185]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(8866, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=256, out_features=18, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [186]:
def count_parameters(model):
    return sum(p.data.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,522,010 trainable parameters


In [187]:
pretrained_embeddings = TEXT.vocab.vectors

In [188]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9261,  2.3049,  0.5502,  ..., -0.3492, -0.5298, -0.1577],
        [-0.5972,  0.0471, -0.2406,  ..., -0.9446, -0.1126, -0.2260],
        [-0.4809,  2.5629,  0.9530,  ...,  0.5278, -0.4588,  0.7294]])

In [189]:
optimizer = optim.Adam(model.parameters())

In [190]:
TAG_PAD_IDX = UP_TAGS.vocab.stoi[UP_TAGS.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TAG_PAD_IDX)

In [191]:
model = model.to(device)
criterion = criterion.to(device)

In [202]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0, NOT 8
    """
    
    max_preds = preds.argmax(dim = 1, keepdim = True)
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(y.device)

In [207]:
def train(model, iterator, criterion, optimizer, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in iterator:
        
        optimizer.zero_grad()
        text = batch.text
        tags = batch.udtags
        predictions = model(text)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        loss = criterion(predictions, tags)
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [208]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [209]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [212]:
N_EPOCHS = 15
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, criterion, optimizer, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    print(f'Epoch: {epoch+1:02} | EPoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss:{train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\Val Loss:{valid_loss:.3f} | Train Acc: {valid_acc*100:.2f}%')

Epoch: 01 | EPoch Time: 0m 3s
	Train Loss:0.287 | Train Acc: 90.89%
\Val Loss:0.400 | Train Acc: 86.68%
Epoch: 02 | EPoch Time: 0m 3s
	Train Loss:0.250 | Train Acc: 92.01%
\Val Loss:0.384 | Train Acc: 86.97%
Epoch: 03 | EPoch Time: 0m 3s
	Train Loss:0.223 | Train Acc: 92.88%
\Val Loss:0.371 | Train Acc: 87.34%
Epoch: 04 | EPoch Time: 0m 3s
	Train Loss:0.204 | Train Acc: 93.45%
\Val Loss:0.364 | Train Acc: 87.62%
Epoch: 05 | EPoch Time: 0m 3s
	Train Loss:0.187 | Train Acc: 94.02%
\Val Loss:0.361 | Train Acc: 87.83%
Epoch: 06 | EPoch Time: 0m 3s
	Train Loss:0.173 | Train Acc: 94.46%
\Val Loss:0.344 | Train Acc: 88.10%
Epoch: 07 | EPoch Time: 0m 3s
	Train Loss:0.163 | Train Acc: 94.72%
\Val Loss:0.345 | Train Acc: 88.09%
Epoch: 08 | EPoch Time: 0m 3s
	Train Loss:0.155 | Train Acc: 95.01%
\Val Loss:0.341 | Train Acc: 88.17%
Epoch: 09 | EPoch Time: 0m 3s
	Train Loss:0.146 | Train Acc: 95.22%
\Val Loss:0.333 | Train Acc: 89.27%
Epoch: 10 | EPoch Time: 0m 3s
	Train Loss:0.139 | Train Acc: 95.

In [215]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'\tTrain Loss:{test_loss:.3f} | Train Acc: {test_acc*100:.2f}%')

	Train Loss:0.362 | Train Acc: 88.93%
